In [9]:
import pandas as pd
import numpy as np
import joblib
import os
import json
import warnings
import tensorflow as tf
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# 불필요한 경고 메시지는 숨겨둘게요
warnings.filterwarnings('ignore')

# 1. 데이터 로드 및 기초 가공
print("딥러닝 데이터 준비 중...")
# 파일 경로를 사용자 환경에 맞춰 불러옵니다
df = pd.read_csv('../data/spotify_churn_dataset.csv')

# 식별자 컬럼은 학습에 방해되니 제거합니다
if 'user_id' in df.columns:
    df = df.drop(columns=['user_id'])

# 피처 엔지니어링: 도메인 지식을 활용해 의미 있는 변수를 추가합니다
df['ad_burden'] = df['ads_listened_per_week'] / (df['listening_time'] + 1)
df['satisfaction_score'] = df['songs_played_per_day'] * (1 - df['skip_rate'])
df['time_per_song'] = df['listening_time'] / (df['songs_played_per_day'] + 1)

X = df.drop(columns=['is_churned'])
y = df['is_churned']

# 2. 데이터셋 분리 (Train 60% / Val 20% / Test 20%)
# 전체에서 먼저 20%를 테스트용으로 떼어놓습니다
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 남은 80%를 다시 75:25로 나누어 최종적으로 60% 학습, 20% 검증 세트를 만듭니다
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val)

# 3. 전처리 파이프라인 구성
numerical_cols = [
    'age', 'listening_time', 'songs_played_per_day', 'skip_rate', 
    'ads_listened_per_week', 'offline_listening',
    'ad_burden', 'satisfaction_score', 'time_per_song'
]
categorical_cols = ['gender', 'country', 'subscription_type', 'device_type']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols)
    ])

# 학습 데이터(Train)를 기준으로 전처리기를 학습시키고 모든 데이터를 변환합니다
X_train_proc = preprocessor.fit_transform(X_train)
X_val_proc = preprocessor.transform(X_val)
X_test_proc = preprocessor.transform(X_test)

# 4. BorderlineSMOTE 적용 (중요: 오직 학습 데이터에만 적용해서 오염을 방지합니다)
print("학습 데이터에만 SMOTE 적용 중... (검증/테스트 데이터는 보존)")
bsmote = BorderlineSMOTE(random_state=42, kind='borderline-1')
X_train_res, y_train_res = bsmote.fit_resample(X_train_proc, y_train)

# 5. 딥러닝 모델 설계 (Sequential DNN)
input_dim = X_train_res.shape[1]
model = Sequential([
    Dense(512, input_dim=input_dim, activation='swish'),
    BatchNormalization(), Dropout(0.4),
    Dense(256, activation='swish'),
    BatchNormalization(), Dropout(0.4),
    Dense(128, activation='swish'),
    BatchNormalization(), Dropout(0.3),
    Dense(64, activation='swish'),
    BatchNormalization(), Dropout(0.3),
    Dense(32, activation='swish'),
    BatchNormalization(), Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])
print(f"\n모델 구조가 준비되었습니다. (입력 차원: {input_dim})")

# 6. 모델 학습 실행
print("학습 시작...")
EPOCHS = 250
early_stop = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0)
checkpoint = ModelCheckpoint('../models/spotify_dl_model.h5', monitor='val_accuracy', save_best_only=True, verbose=0)

history = model.fit(
    X_train_res, y_train_res, 
    epochs=EPOCHS, 
    batch_size=64, 
    validation_data=(X_val_proc, y_val), # 분리해둔 검증 데이터를 직접 사용합니다
    callbacks=[early_stop, reduce_lr, checkpoint], 
    verbose=1
)

# 7. 최적의 임계값(Threshold) 찾기
print("\n최종 테스트 데이터로 최적 임계값 탐색 중...")
best_model = tf.keras.models.load_model('../models/spotify_dl_model.h5')
y_pred_proba = best_model.predict(X_test_proc).flatten()

thresholds = np.arange(0.3, 0.7, 0.01)
best_thresh, best_f1, best_acc = 0.5, 0, 0

for thresh in thresholds:
    y_pred_temp = (y_pred_proba >= thresh).astype(int)
    f1_temp = f1_score(y_test, y_pred_temp)
    
    if f1_temp > best_f1:
        best_f1 = f1_temp
        best_acc = accuracy_score(y_test, y_pred_temp)
        best_thresh = thresh

print("-" * 45)
print(f"🚀 최적 임계값 발견: {best_thresh:.2f}")
print(f"최종 Accuracy: {best_acc:.4f}")
print(f"최종 F1-Score: {best_f1:.4f}")
print("-" * 45)

# 8. 결과 및 전처리기 저장
if not os.path.exists('../models'): os.makedirs('../models')
joblib.dump(preprocessor, '../models/dl_preprocessor.pkl')

metrics_file = '../data/model_metrics.json'
final_metrics = {}

if os.path.exists(metrics_file):
    try:
        with open(metrics_file, 'r') as f:
            final_metrics = json.load(f)
    except:
        pass

# 딥러닝 성능 지표 업데이트
final_metrics['Deep Learning (DNN)'] = {
    'Accuracy': best_acc, 
    'F1-Score': best_f1,
    'Best Threshold': float(best_thresh)
}

with open(metrics_file, 'w') as f:
    json.dump(final_metrics, f, indent=4)

print("딥러닝 모델 및 최종 성능 지표 저장 완료.")

딥러닝 데이터 준비 중...
학습 데이터에만 SMOTE 적용 중... (검증/테스트 데이터는 보존)

모델 구조가 준비되었습니다. (입력 차원: 20)
학습 시작...
Epoch 1/250
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5918 - loss: 0.7713

128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.6446 - loss: 0.6914 - val_accuracy: 0.4990 - val_loss: 0.6779 - learning_rate: 5.0000e-04
Epoch 2/250
115/128 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7236 - loss: 0.5676

128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7221 - loss: 0.5753 - val_accuracy: 0.6220 - val_loss: 0.6311 - learning_rate: 5.0000e-04
Epoch 3/250
119/128 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7432 - loss: 0.5388

128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7479 - loss: 0.5304 - val_accuracy: 0.7210 - val_loss: 0.5473 - learning_rate: 5.0000e-04
Epoch 4/250
121/128 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7744 - loss: 0.5060

128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7655 - loss: 0.5152 - val_accuracy: 0.7440 - val_loss: 0.5172 - learning_rate: 5.0000e-04
Epoch 5/250
116/128 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7622 - loss: 0.5041

...

128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8056 - loss: 0.4395 - val_accuracy: 0.7670 - val_loss: 0.4755 - learning_rate: 2.5000e-04
Epoch 24/250
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8065 - loss: 0.4371 - val_accuracy: 0.7620 - val_loss: 0.4863 - learning_rate: 2.5000e-04
Epoch 25/250
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8097 - loss: 0.4347 - val_accuracy: 0.7645 - val_loss: 0.4812 - learning_rate: 2.5000e-04
Epoch 26/250
122/128 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8127 - loss: 0.4304

128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8069 - loss: 0.4358 - val_accuracy: 0.7675 - val_loss: 0.4792 - learning_rate: 2.5000e-04
Epoch 27/250
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8056 - loss: 0.4314 - val_accuracy: 0.7665 - val_loss: 0.4814 - learning_rate: 2.5000e-04
Epoch 28/250
124/128 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8104 - loss: 0.4303

128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8091 - loss: 0.4305 - val_accuracy: 0.7690 - val_loss: 0.4849 - learning_rate: 2.5000e-04
Epoch 29/250
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8009 - loss: 0.4376 - val_accuracy: 0.7685 - val_loss: 0.4812 - learning_rate: 2.5000e-04
Epoch 30/250
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8115 - loss: 0.4291 - val_accuracy: 0.7685 - val_loss: 0.4827 - learning_rate: 2.5000e-04
Epoch 31/250
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8072 - loss: 0.4320 - val_accuracy: 0.7685 - val_loss: 0.4847 - learning_rate: 2.5000e-04
Epoch 32/250
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8096 - loss: 0.4280 - val_accuracy: 0.7680 - val_loss: 0.4910 - learning_rate: 1.2500e-04
Epoch 33/250
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8098 - loss: 0.4281 - val_accuracy: 0.7655 - val_loss: 0.4882 - learning_rate: 1.2500e-04
Epoch 34/250
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy:

128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8103 - loss: 0.4199 - val_accuracy: 0.7695 - val_loss: 0.4891 - learning_rate: 1.2500e-04
Epoch 39/250
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8105 - loss: 0.4225 - val_accuracy: 0.7690 - val_loss: 0.4915 - learning_rate: 1.2500e-04
Epoch 40/250
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8132 - loss: 0.4238 - val_accuracy: 0.7680 - val_loss: 0.4913 - learning_rate: 1.2500e-04
Epoch 41/250
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8121 - loss: 0.4223 - val_accuracy: 0.7690 - val_loss: 0.4909 - learning_rate: 1.2500e-04

최종 테스트 데이터로 최적 임계값 탐색 중...


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
---------------------------------------------
🚀 최적 임계값 발견: 0.67
최종 Accuracy: 0.8105
최종 F1-Score: 0.7413
---------------------------------------------
딥러닝 모델 및 최종 성능 지표 저장 완료.
